In [1]:
import xmltodict
import pandas as pd
import copy
import csv
import os
import langid
import mysql.connector
import re

from IPython.display import clear_output
BASE_DIR = '/home/antonio/lattes_base/Collection'

In [2]:
def insert_values(doc, cursor):
    try:
        author_id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
        print(author_id)
        name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-COMPLETO']
        citation_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-EM-CITACOES-BIBLIOGRAFICAS']

        try:
            try:
                graduacao = doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['GRADUACAO']['@NOME-CURSO']
            except:
                graduacao = doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['GRADUACAO'][0]['@NOME-CURSO']
        except:
            graduacao = None
        try:
            mestrado = doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['MESTRADO']['@NOME-CURSO']
        except:
            mestrado = None
        try:
            doutorado = doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['DOUTORADO']['@NOME-CURSO']
        except:
            doutorado = None

        sql = "INSERT INTO tbl_Autor (IdAutor, NomeAutor, NomeCitacao, Graduacao, Mestrado, Doutorado) VALUES (%s,%s,%s,%s,%s,%s)"
        values = (author_id, name, citation_name, graduacao, mestrado, doutorado)
        cursor.execute(sql, values)


        try:
            artigos = doc['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
            for artigo in artigos:
                if(artigo['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] == "Inglês"):
                    try:
                        issn = artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN']
                        titulo = artigo['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO']
                        ano = artigo['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO']
                        home = artigo['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO']
                        sql = "INSERT INTO tbl_Artigos (Issn, Titulo, Ano, Homepage) VALUES (%s,%s,%s,%s)"
                        values = (issn, titulo, ano, home)
                        cursor.execute(sql, values)
                    except:
                        pass
        except:
            pass
    except:
        pass

In [3]:
def link_papers_authors(doc, cursor):
    author = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
    artigos = doc['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
    for artigo in artigos:
        if(artigo['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] == "Inglês"):
            sql = "INSERT INTO tbl_ArtigosAutores (IdAutor, Issn,) VALUES (%s,%s)"
            values = (author, artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN'])
            cursor.execute(sql, values)

In [4]:
def link_coauthors(doc, cursor):
    artigos = doc['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
    for artigo in artigos:
        if(artigo['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] == "Inglês"):
            issn = artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN']
            for autores in artigo['AUTORES']:     
                sql = "INSERT INTO tbl_ArtigosAutores (IdAutor, Issn,) VALUES (%s,%s)"
                values = (autores['@NRO-ID-CNPQ'], issn)
                cursor.execute(sql, values)

### get all files

In [5]:
files_list = []

for folder in os.listdir(BASE_DIR):
    path = os.path.join(BASE_DIR, folder)
    if(os.path.isdir(path) == True and not folder.startswith('.')):
        for files in os.listdir(path):
            if not files.startswith('.'):
                file = os.path.join(path, files)
                files_list.append(file)


In [6]:
n_files = len(files_list)
n_files

295303

### Define tasks

In [7]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="lattes"
)

In [8]:
cursor = mydb.cursor()

In [ ]:
for index, file in enumerate (files_list, start=0):
    with open(file + '/curriculo.xml', encoding="ISO-8859-1") as fd:
        doc = xmltodict.parse(fd.read())
        insert_values(doc, cursor)
        clear_output(wait=True)
        print(str(index * 100 / n_files) + '%')

3.0717601920739037%


In [ ]:
for index, file in enumerate (files_list, start=0):
    with open(file + '/curriculo.xml', encoding="ISO-8859-1") as fd:
        doc = xmltodict.parse(fd.read())
        link_papers_authors(doc, cursor)
        clear_output(wait=True)
        print(str(index * 100 / n_files) + '%')        

In [ ]:
for index, file in enumerate (files_list, start=0):
    with open(file + '/curriculo.xml', encoding="ISO-8859-1") as fd:
        doc = xmltodict.parse(fd.read())
        link_coauthors(doc, cursor)
        clear_output(wait=True)
        print(str(index * 100 / n_files) + '%')

In [ ]:
cursor.close()
mydb.commit()
mydb.close()